<a href="https://colab.research.google.com/github/myrjybyn/CSST-101-3B-AY2526/blob/main/Assessment_Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Unification Checker**

In [2]:
def unify(p1, p2):
    f1, f2 = p1.split("(")[0], p2.split("(")[0]
    if f1 != f2:
        return "Cannot be unified"

    a1, a2 = p1[p1.find("(")+1:-1].split(","), p2[p2.find("(")+1:-1].split(",")
    if len(a1) != len(a2):
        return "Cannot be unified"

    subs = {}
    for x, y in zip(a1, a2):
        x, y = x.strip(), y.strip()
        if x == y: continue
        if x.islower(): subs[x] = y
        elif y.islower(): subs[y] = x
        else: return "Cannot be unified"

    return subs or "Already unified"


# Part 1 Test
print("Unification Result:", unify("Eligible(x)", "Eligible(Ana)"))


Unification Result: {'x': 'Ana'}


# **2. Forward Chaining**

In [4]:
facts = {
    "Juan": {"HighGrades", "Applied"},
    "Maria": {"HighGrades", "DidNotApply"},
    "Carlos": {"StudentLeader", "NoHighGrades"},
    "Ana": {"HighGrades", "StudentLeader", "Applied"}
}

derived = {name: set() for name in facts}

def apply_rules(name, props):
    new = set()
    # Rule 1: HighGrades + StudentLeader + Applied -> Eligible
    if {"HighGrades", "StudentLeader", "Applied"}.issubset(props):
        new.add("Eligible")
    # Rule 2: Eligible -> Scholarship
    if "Eligible" in props:
        new.add("Scholarship")
    # Rule 3: Scholarship -> FinancialSupport
    if "Scholarship" in props:
        new.add("FinancialSupport")
    return new


changed = True
while changed:
    changed = False
    for student, props in facts.items():
        new = apply_rules(student, props | derived[student])
        if not new.issubset(derived[student]):
            derived[student].update(new)
            changed = True

for student in facts:
    print("Final Facts: ")
    print(student, "→", facts[student] | derived[student])


Final Facts: 
Juan → {'Applied', 'HighGrades'}
Final Facts: 
Maria → {'DidNotApply', 'HighGrades'}
Final Facts: 
Carlos → {'NoHighGrades', 'StudentLeader'}
Final Facts: 
Ana → {'StudentLeader', 'Scholarship', 'Eligible', 'Applied', 'FinancialSupport', 'HighGrades'}




*   New Facts (Juan): None
*   New Facts(Ana): Eligible, Scholarship, Financial Support


---



*   Maria is not Eligible because she's not a student leader and did not apply for the scholarship. Carlos is not eligible because he doesn't have high grades.





# **3. Backward Chaining**

In [5]:
facts = {
    "Juan": {"HighGrades", "Applied"},
    "Maria": {"HighGrades", "DidNotApply"},
    "Carlos": {"StudentLeader", "NoHighGrades"},
    "Ana": {"HighGrades", "StudentLeader", "Applied"}
}

# Backward chaining function
def prove(goal, student):

    if goal in facts.get(student, set()):
        return True, f"{goal}({student}) is a given fact."

    # Rule 1
    if goal == "Eligible":
        if {"HighGrades", "StudentLeader", "Applied"}.issubset(facts[student]):
            return True, f"{goal}({student}) proven by Rule 1 (has HighGrades, StudentLeader, and Applied)."
        else:
            return False, f"{goal}({student}) fails Rule 1 (missing one or more requirements)."

    # Rule 2
    if goal == "Scholarship":
        ok, reason = prove("Eligible", student)
        if ok:
            return True, f"{goal}({student}) proven by Rule 2 → {reason}"
        else:
            return False, f"{goal}({student}) fails because {reason}"

    # Rule 3
    if goal == "FinancialSupport":
        ok, reason = prove("Scholarship", student)
        if ok:
            return True, f"{goal}({student}) proven by Rule 3 → {reason}"
        else:
            return False, f"{goal}({student}) fails because {reason}"

    return False, f"No rule or fact supports {goal}({student})."


# Tests
print("Goal 1:", prove("Scholarship", "Maria")[1])
print("Goal 2:", prove("FinancialSupport", "Ana")[1])


Goal 1: Scholarship(Maria) fails because Eligible(Maria) fails Rule 1 (missing one or more requirements).
Goal 2: FinancialSupport(Ana) proven by Rule 3 → Scholarship(Ana) proven by Rule 2 → Eligible(Ana) proven by Rule 1 (has HighGrades, StudentLeader, and Applied).


**Goal 1: Scholarship(Maria)**


*   Facts: HighGrades(Maria), DidNotApply(Maria)
*   Rule 2 says if Eligible(x) then, Scholarship(x)
*   Subgoal: Scholarship(Maria)
*   However, Rule 1 says HighGrades(x) ∧ StudentLeader(x) ∧ Applied(x), then Eligible(x)
*   **Scholarship(Maria) can't be proven or not possible since Maria lacks the student leader attribute and she did not apply.**


---


**Goal 2: FinancialSupport(Ana)**
*   Facts: HighGrades(Ana), StudentLeader(Ana), Applied(Ana)
*   Rule 1 says HighGrades(x) ∧ StudentLeader(x) ∧ Applied(x), then Eligible(x)
*   Subgoal: Eligible(Ana)
*   Rule 2 says if Eligible(x), then Scholarship(x)
*   Subgoal: Scholarship(Maria)
*   Rule 3 says if Scholarship(x), then FinancialSupport(x)
*   **FinancialSupport(Ana) is proven true because Ana has high grades, a student leader and applied for the scholarship.**










# **Part 4: Truth Table**
| **HighGrades** | **Leader** | **Applied** | **HG∧L∧A** |    **Eligible**    |
|:--------------:|:----------:|:-----------:|:----------:|:------------------:|
|        T       |      T     |      T      |      T     |          T         |
|        T       |      T     |      F      |      F     | Vacuously True (T) |
|        T       |      F     |      T      |      F     | Vacuously True (T) |
|        T       |      F     |      F      |      F     | Vacuously True (T) |
|        F       |      F     |      F      |      F     | Vacuously True (T) |
|        F       |      F     |      T      |      F     | Vacuously True (T) |
|        F       |      T     |      T      |      F     | Vacuously True (T) |
|        F       |      T     |      F      |      F     | Vacuously True (T) |


---
# **Test**
**Juan**
* Facts: {HighGrades, Applied} → Missing Leader → antecedent is False → implication is True, but Juan is not Eligible.  

**Maria**
* Facts: {HighGrades, DidNotApply} → Missing Applied and Leader → antecedent is False → implication holds, but Maria is not Eligible.  

**Carlos**
* Facts: {StudentLeader, NoHighGrades} → Missing HighGrades → antecedent is False → implication holds, but Carlos is not Eligible.  

**Ana**
* Facts: {HighGrades, StudentLeader, Applied} → All True → antecedent is True → therefore Eligible(Ana) = True.

# **Part5: Reflection**  
Logical reasoning is the foundation of AI decision making. It allows AI to make rule-based decisions, making it consistent. It enables AI to think like humans when it comes to reasoning. By following clear reasoning steps, AI justify its conclusions. In real life application, it helps avoid bias.




